In [62]:
    %autosave 60


Autosaving every 60 seconds


In [63]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [64]:
df = pd.read_csv("df.csv")

In [65]:
df=df.rename(columns={"Tota_Score":"Total_Score"})

In [66]:
df

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,Average,Total_Score
0,female,group B,bachelor's degree,standard,none,72,72,74,72.67,218
1,female,group C,some college,standard,completed,69,90,88,82.33,247
2,female,group B,master's degree,standard,none,90,95,93,92.67,278
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.33,148
4,male,group C,some college,standard,none,76,78,75,76.33,229
...,...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,94.00,282
996,male,group C,high school,free/reduced,none,62,55,55,57.33,172
997,female,group C,high school,free/reduced,completed,59,71,65,65.00,195
998,female,group D,some college,standard,completed,68,78,77,74.33,223


In [67]:
X=df.drop(["Total_Score","math_score","Average"], axis=1)

In [68]:
y=df["math_score"]

In [69]:
X

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75
...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,99,95
996,male,group C,high school,free/reduced,none,55,55
997,female,group C,high school,free/reduced,completed,71,65
998,female,group D,some college,standard,completed,78,77


In [70]:
nan_features = [feature for feature in X.columns if X[feature].isna().any()]
categorical_features = [feature for feature in X.columns if X[feature].dtype == 'O']
year_features = [feature for feature in X.columns if "Year" in feature or "Yr" in feature]
numerical_features = [feature for feature in X.columns if X[feature].dtype != 'O']
discrete_features = [feature for feature in numerical_features if len(X[feature].unique()) <= 25]
continous_features = [feature for feature in numerical_features if len(X[feature].unique()) > 25 and feature not in year_features+["Id"]]

In [71]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

ohe = OneHotEncoder()
sc = StandardScaler()
ct = ColumnTransformer(
    [
        ("OneHotEncoder",ohe,categorical_features),
        ("StandardScaler",sc,numerical_features)
    ]
)

In [72]:
X = ct.fit_transform(X)

In [74]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [75]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 42)

In [76]:
X_train.shape

(800, 19)

In [78]:
X_train

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.05694554,  0.45733301],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.94779033,  0.98406266],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.35894946,  1.18158627],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
        -0.49126664, -0.99117351],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
        -1.45063795, -0.99117351],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.4960025 ,  1.37910989]])

In [79]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
def evaluate_model(true,predicted):
  mae=mean_absolute_error(true,predicted)
  mse=mean_squared_error(true,predicted)
  rmse= np.sqrt(mean_squared_error(true,predicted))
  r2_square=r2_score(true,predicted)
  #accuracy = accuracy_score(true,predicted)
  #f1 = f1_score(true,predicted,average='weighted')
  return mae,rmse,r2_square #,accuracy,f1

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, ARDRegression
from sklearn.metrics._plot.precision_recall_curve import precision_recall_curve
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve
from xgboost import XGBRegressor



r2_list=[]
models = {
    "Linear Regression" : LinearRegression(),
    "XGB Regression": XGBRegressor()
}

for name, model in models.items():

  # Nfittiw el modele mte3na
  model.fit(X_train, y_train)

  # Npredictiw el training wel testing
  predicted_y_train = model.predict(X_train)
  predicted_y_test = model.predict(X_test)

  # Ne7sbou el performance tawa mtaa el training set
  mae_train,rmse_train,r2_train = evaluate_model(y_train,predicted_y_train)
  #model_train_accuracy = accuracy_score(y_train,predicted_y_train)
  #model_train_f1 = f1_score(y_train,predicted_y_train,average='weighted')
  #r2 = precision_score(y_train,predicted_y_train)
  #model_train_recall = recall_score(y_train,predicted_y_train)
  #model_train_roc_auc = roc_auc_score(y_train,predicted_y_train)


  # Ne7sbou el performance tawa mtaa el testing set
  mae_test,rmse_test,r2_test = evaluate_model(y_test,predicted_y_test)
  #model_test_accuracy = accuracy_score(y_test,predicted_y_test)
  #model_test_f1 = f1_score(y_test,predicted_y_test,average='weighted')
  #model_test_precision = precision_score(y_test,predicted_y_test)
  #model_test_recall = recall_score(y_test,predicted_y_test)
  #model_test_roc_auc = roc_auc_score(y_test,predicted_y_test)

  # Print the performances
  print("*"*38)
  print(f"*  --> {list(models.keys())[i]} ")
  print("*                                    *")
  print("* Model Performance for Training set *")
  print("* - MAE: {:.4f}                 *".format(mae_train))
  print("* - RMSE: {:.4f}                 *".format(rmse_train))
  print("* - R2: {:.4f}                *".format(r2_train))
  #print("* - Accuracy: {:.4f}                   *".format(accuracy_train))
  #print("* - F1: {:.4f}                  *".format(f1_train))


  print("*                                    *")
  print("* Model Performance for Testing set  *")
  print("* - MAE: {:.4f}                 *".format(mae_test))
  print("* - RMSE: {:.4f}                 *".format(rmse_test))
  print("* - R2: {:.4f}                *".format(r2_test))
  #print("* - Accuracy: {:.4f}                   *".format(accuracy_test))
  #print("* - F1: {:.4f}                  *".format(f1_test))

  r2_list.append(r2_test)

  print("*"*38)
  print("\n")


**************************************
*  --> Linear Regression 
*                                    *
* Model Performance for Training set *
* - MAE: 4.2667                 *
* - RMSE: 5.3231                 *
* - R2: 0.8743                *
*                                    *
* Model Performance for Testing set  *
* - MAE: 4.2148                 *
* - RMSE: 5.3940                 *
* - R2: 0.8804                *
**************************************


**************************************
*  --> Linear Regression 
*                                    *
* Model Performance for Training set *
* - MAE: 0.6875                 *
* - RMSE: 1.0073                 *
* - R2: 0.9955                *
*                                    *
* Model Performance for Testing set  *
* - MAE: 5.0577                 *
* - RMSE: 6.4733                 *
* - R2: 0.8278                *
**************************************




In [88]:
pd.DataFrame(list(zip(models, r2_list)), columns=["Model Name3", "R2_SCORE"]).sort_values(by=["R2_SCORE"],ascending=False)

,Model Name3,R2_SCORE
0,Linear Regression,0.880433
1,XGB Regression,0.827797


In [92]:
lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_test)

In [95]:
pd.DataFrame({'Actual Value': y_test, 'Predicted Value':y_pred, 'Difference':y_test-y_pred})

,Actual Value,Predicted Value,Difference
521,91,76.387970,14.612030
737,53,58.885970,-5.885970
740,80,76.990265,3.009735
660,74,76.851804,-2.851804
411,84,87.627378,-3.627378
...,...,...,...
408,52,43.409149,8.590851
332,62,62.152214,-0.152214
208,74,67.888395,6.111605
613,65,67.022287,-2.022287
